In [1]:
!pip install dagshub mlflow --quiet

import warnings
warnings.filterwarnings("ignore")

print("Done!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 53.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [3]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import mlflow
import mlflow.sklearn
import dagshub
import joblib
import os
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

mlflow.set_experiment("XGBoost_Training")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d90c47f0-b604-45e3-83a1-652dd9c2efac&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=29f6b76d5ba84a0362827bf46c3c2be1d3c14c5e95e569f0d3d6d2da005c7a4b




Output()

Accessing as gnada22

Initialized MLflow to track repo "gnada22/ml_final_project"

Repository gnada22/ml_final_project initialized!

<Experiment: artifact_location='mlflow-artifacts:/62e2117acafa4300989c6ce37ac3c340', creation_time=1751448912184, experiment_id='1', last_update_time=1751448912184, lifecycle_stage='active', name='XGBoost_Training', tags={}>

In [13]:
class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["week"] = (
            X["Date"].dt.to_period("W")
            .rank(method="dense")
            .astype(int) - 1
        )
        X["sin_13"] = np.sin(2 * np.pi * X["week"] / 13)
        X["cos_13"] = np.cos(2 * np.pi * X["week"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["week"] / 23)
        X["cos_23"] = np.cos(2 * np.pi * X["week"] / 23)
        X = X.drop(columns=["Date"])
        return X

date_features = ["week", "sin_13", "cos_13", "sin_23", "cos_23"]

class LagFeatureAdder:
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.sort_values(["Store", "Dept", "Date"])
        df["lag_1"] = df.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1)
        df["lag_52"] = df.groupby(["Store", "Dept"])["Weekly_Sales"].shift(52)
        return df

lag_features = ["lag_1", "lag_52"]

added_features = date_features + lag_features

class ColumnTransformerWithNames(ColumnTransformer):
    def get_feature_names_out(self, input_features=None):
        return super().get_feature_names_out(input_features)

    def transform(self, X):
        X_transformed = super().transform(X)
        # Get feature names for columns
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        # print("with name transform - ", type(res))
        return res

    def fit_transform(self, X, y=None):
        X_transformed = super().fit_transform(X, y)
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        # print("with name fit_transform - ", type(res))
        return res

In [ ]:
class ResidualRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, base_model, residual_model):
        self.base_model = base_model
        self.residual_model = residual_model

    def fit(self, X, y):
        # print("F - ", type(X))
        # X = pd.DataFrame(X)
        self.base_model_ = clone(self.base_model)
        self.base_model_.fit(X, y)
        residuals = y - self.base_model_.predict(X)
        
        self.residual_model_ = clone(self.residual_model)
        self.residual_model_.fit(X, residuals)
        return self

    def predict(self, X):
        print("P - ", type(X))
        print(X.columns)
        # print(X)
        # X = pd.DataFrame(X)
        pred_lookup = {}
        weekly_preds = pd.Series(index=X.index, dtype=float)
        
        for week in tqdm(X["week"].sort_values().unique(), desc="Recursive prediction"):
            day_rows = X[X["week"] == week].copy()
        
            for idx, row in day_rows.iterrows():
                store_cols = [col for col in row.index if col.startswith("Store_")]
                store = int([col for col in store_cols if row[col] == 1][0].split("_")[1])
                
                dept_cols = [col for col in row.index if col.startswith("Dept_")]
                dept = int([col for col in dept_cols if row[col] == 1][0].split("_")[1])
        
                # Get keys for previous lags
                key_1 = (store, dept, week - 1)
                key_52 = (store, dept, week - 52)
        
                lag_1 = pred_lookup.get(key_1, row["lag_1"])
                lag_52 = pred_lookup.get(key_52, row["lag_52"])
        
                day_rows.at[idx, "lag_1"] = lag_1
                day_rows.at[idx, "lag_52"] = lag_52
        
            # Predict all rows for this day in one batch
            y_preds = self.pred_f(day_rows)
        
            # Assign predictions back
            weekly_preds[day_rows.index] = y_preds
            
            # Update lookup for future lag access
            for idx, pred in zip(day_rows.index, y_preds):
                row = day_rows.loc[idx]
                
                store_cols = [col for col in row.index if col.startswith("Store_")]
                store = int([col for col in store_cols if row[col] == 1][0].split("_")[1])
                
                dept_cols = [col for col in row.index if col.startswith("Dept_")]
                dept = int([col for col in dept_cols if row[col] == 1][0].split("_")[1])
                
                key = (store, dept, row["week"])
                pred_lookup[key] = pred
    
        return weekly_preds.fillna(0).to_numpy()

    def pred_f(self, X):
        return self.base_model_.predict(X) + self.residual_model_.predict(X)

In [6]:
# Load and merge data
df_train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
df_features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
df_stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = df_train.merge(df_features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(df_stores, on="Store", how="left")

In [23]:
categorical = ["Store", "Dept", "Type", "IsHoliday"]
numerical = ["Temperature", "Fuel_Price", "CPI", "Unemployment",
             "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]
engineered = added_features

categorical_transformer = OneHotEncoder(handle_unknown="ignore")
numerical_transformer = SimpleImputer(strategy="mean")

preprocessor = ColumnTransformerWithNames(
    transformers=[
        ("num", numerical_transformer, numerical + engineered),
        ("cat", categorical_transformer, categorical),
    ],
    sparse_threshold=0.0
)

residual_model = ResidualRegressor(
    base_model=LinearRegression(),
    residual_model=XGBRegressor(n_estimators=100, max_depth=3)
)

pipeline = Pipeline([
    ("date_features", DateFeatureCreator()),
    ("preprocessor", preprocessor),
    ("regressor", residual_model)
])

param_grid = {
    "regressor__residual_model__n_estimators": [50, 100, 200],
    "regressor__residual_model__max_depth": [3, 5, 7],
    "regressor__residual_model__learning_rate": [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1,
    verbose=1
)

In [8]:
print(df.columns)

with mlflow.start_run(run_name="Feature_Engineering"):
    lag_adder = LagFeatureAdder()
    df = lag_adder.transform(df)
    original_rows = len(df)
    df = df.dropna(subset=["lag_1", "lag_52"])
    remaining_rows = len(df)
    mlflow.log_param("rows_before_dropna", original_rows)
    mlflow.log_param("rows_after_dropna", remaining_rows)
    mlflow.log_param("features_added", added_features)

y = df["Weekly_Sales"]
X = df.drop(columns=["Weekly_Sales"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size'],
      dtype='object')
🏃 View run Feature_Engineering at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/1/runs/8f5961e9055148cf8ba1011002e0494e
🧪 View experiment at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/1


In [25]:
# print(X_train.columns)

with mlflow.start_run(run_name="Training"):
    # grid_search.fit(X_train, y_train)
    # best_model = grid_search.best_estimator_
    # preds = best_model.predict(X_val)

    # mlflow.log_params(grid_search.best_params_)

    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_val)
    best_model = pipeline

    mae = mean_absolute_error(y_val, preds)
    weights = X_val["IsHoliday"].apply(lambda x: 5 if x else 1)
    wmae = (weights * np.abs(y_val - preds)).sum() / weights.sum()

    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("WMAE", wmae)
    # mlflow.sklearn.log_model(best_model, artifact_path="model")

    model_path = "model.pkl"
    joblib.dump(best_model, model_path)
    mlflow.log_artifact(model_path)

P -  <class 'pandas.core.frame.DataFrame'>
Index(['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown1',
       'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'week',
       ...
       'Dept_95', 'Dept_96', 'Dept_97', 'Dept_98', 'Dept_99', 'Type_A',
       'Type_B', 'Type_C', 'IsHoliday_False', 'IsHoliday_True'],
      dtype='object', length=142)


Recursive prediction: 100%|██████████| 91/91 [00:39<00:00,  2.29it/s]


🏃 View run Training at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/1/runs/61e2644fb47f4ef69d86fd860e32dc6c
🧪 View experiment at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/1
